### ESP-01s 模組
![ESP-01s IOT-MCU DHT11](iot-mcu_dht11.jpg)

In [ ]:
%serialconnect to --port COM5

In [36]:
%sendtofile boot.py
# This file is executed on every boot (including wake-boot from deepsleep)
#import esp
#esp.osdebug(None)
import uos, machine
#uos.dupterm(None, 1) # disable REPL on UART(0)
import gc
gc.collect()

Sent 7 lines (196 bytes) to boot.py.


In [35]:
%sendtofile boot.py
# This file is executed on every boot (including wake-boot from deepsleep)
#import esp
#esp.osdebug(None)
import uos, machine
#uos.dupterm(None, 1) # disable REPL on UART(0)
import gc
import network
import time
SSID='BUFFALO-2387202'
WKEY='0953313123'
sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)
sta_if.connect(SSID,WKEY)
time.sleep(2)
print(sta.isconnected())
print(sta_if.ifconfig())
#import webrepl
#webrepl.start()
gc.collect()

Sent 19 lines (445 bytes) to boot.py.


Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'O' isn't defined


In [25]:
%sendtofile main.txt
# -*- coding: utf-8 -*-

import machine
import network
import time
import os

#- check ap config file
AP_SSID = 'upy'
AP_PWD = 'pypypypy'
ap_config = None
ap_config_fn = 'ap.txt'
if ap_config_fn in os.listdir():
    print('ap config here!')
    f = open(ap_config_fn)
    ap_config = f.read()
    f.close()
if ap_config:
    print( ('ap_config:', ap_config))
    ap_config = ap_config.split('\n')
    AP_SSID = ap_config[0].strip()
    AP_PWD = ap_config[1].strip()
print('line to: ', (AP_SSID, AP_PWD))

#-- 連到AP 為Station
sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)
sta_if.connect(AP_SSID, AP_PWD)
sta_if.isconnected()
for i in range(20):
    time.sleep(0.5)
    if sta_if.isconnected():
        break
sta_ip = ''
if sta_if.isconnected():
    print('connected!  --> ', sta_if.ifconfig())
    sta_ip = sta_if.ifconfig()[0]
else:
    print('not connected!  --> ', sta_if.ifconfig())

#-- 當AP，並指定
uid = machine.unique_id()
#ap_if.ifconfig()
# ('192.168.4.1', '255.255.255.0', '192.168.4.1', '192.168.43.1')
# (ip, mask, gateway, dns)
my_sn = '%02X-%02X-%02X-%02X' %(uid[0], uid[1], uid[2], uid[3])

#- Change name/password/ip of ESP8266's AP:
ap_if = network.WLAN(network.AP_IF)
#ap_if.ifconfig([my_ip, my_mask, my_gw, my_dns])

my_ssid = 'upy_%s_%s' %(my_sn, sta_ip)
#ap_if.config(essid = my_ssid)#改ssid，馬上生效
ap_if.config(essid=my_ssid, authmode=network.AUTH_WPA_WPA2_PSK, password="12345678")
#DHCP 功能micropython預設就有，不用設定
#AP的IP，每次重開都會回到預設值，因此需要開機時就設定
#一般是配給AP ip的下一號ip


import socket
from machine import Pin
from machine import PWM
import dht
#from hcsr04 import HCSR04


# PIN Define:

D3 = 0  #PWM
D4 = 2  #PWM, #Led on board


#Setup PINS
led = machine.Pin(0, machine.Pin.OUT)


# th_sensor
th_sensor = dht.DHT11(Pin(2))


#Setup Socket WebServer
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('0.0.0.0', 80))
s.listen(5)
while True:
    conn, addr = s.accept()
    print("Got a connection from %s" % str(addr))
    request = conn.recv(1024)
    print("Content = %s" % str(request))

    request = str(request)
    led_on = request.find('GET /?LED=ON')
    led_off = request.find('GET /?LED=OFF')
    th_sensor_read = request.find('GET /?th_sensor=read')


    if led_on >= 0:
        print('TURN Led ON')
        led.value(0)
    if led_off >= 0:
        print('TURN Led OFF')
        led.value(1)

    my_t = None
    if th_sensor_read >= 0:
        print('th_sensor read')
        th_sensor.measure()
        my_t = th_sensor.temperature()
        print('T=', th_sensor.temperature())
        print('H=', th_sensor.humidity())

    f = open('webtool.html')

    while(1):
        html = f.read(1024)

        if my_t:
            html = html.replace('T=?degree', 'T=%d degree' %(my_t))
        else:
            html = html.replace('T=?degree', '')
        #conn.send(html)
        conn.sendall(html) #改用send all就不會有資料傳一半的問題
        if(len(html)<=0):
            break
    f.close()
    conn.close()

Sent 126 lines (2936 bytes) to main.txt.


In [26]:
%sendtofile webtool.html
<!DOCTYPE html>
<html>
<head> <title>upy webserver</title> </head>
<center><h2>upy webserver</h2></center>
<h3>
webREPL: <a href="http://micropython.org/webrepl/" target="_blank">webrepl</a>
<br>
Doc: <a href="https://docs.micropython.org/en/latest/esp8266/esp8266/quickref.html" target="_blank">Quick reference</a>
</h3>
<hr>

<form>
<b>LED (D3):</b><br>
<button name="LED" value="ON" type="submit">ON</button><br>
<button name="LED" value="OFF" type="submit">OFF</button><br>
</form>

<form>
<b>T/H sensor (D4):</b><br>
<button name="th_sensor" value="read" type="submit">Read</button><br>
T=?degree
H=?unit
</form>

<hr>
GPIO Info. :<br>
<pre>

D3 = 0  #PWM
D4 = 2  #PWM, #Led on board

</pre>

</html>

Sent 34 lines (705 bytes) to webtool.html.


In [27]:
%sendtofile ap.txt
BUFFALO-2387202
0953313123

Sent 2 lines (26 bytes) to ap.txt.


In [28]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.10-8-g8b7039d7d on 2019-01-26; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0xbf05c30, open=True>(port='COM6', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [41]:
f = open('ap.txt')
print(f.read())

BUFFALO-238720
0953313123



In [37]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.10-8-g8b7039d7d on 2019-01-26; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft rebooOSError: [Errno 2] ENOENT\r\nMicroPython v1.10-8-g8b7039d7d on 2019-01-26; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [ ]:
%sendtofile blink.py
from machine import Pin
import time

p2 = Pin(0, Pin.OUT)

for i in range(3):
    p2.value(0)
    time.sleep(1)
    p2.value(1)
    time.sleep(1)